# Convert DICOM + RTStruct to NIfTI mask


## Import required functions

In [18]:
import platipy
import os
import re
import shutil
import pandas as pd
import numpy as np
from pathlib import Path
from platipy.dicom.io.rtstruct_to_nifti import convert_rtstruct, read_dicom_struct_file, fix_missing_data

## Define the RTSTRUCT and MR image paths

The image series is required as the output masks will be generated in the same image space as this series.

In [20]:
img_folder = 'img_path'
roi_folder = 'roi_path'
output_folder = 'output_path'

## Convert the RTSTRUCT

Call the convert_rtstruct function to generate the contours.

Optional settings defining the prefix for the mask files generated and the output directory are also passed in to the function.

In [23]:
def convert(file, patient: "Anonymous", dcm_path, rts_path, output_folder):
    for dcm in os.listdir(dcm_path):
        get = int(re.split('_|-',dcm)[0])
        if get == file:
            dcm_path = os.path.join(dcm_path,dcm,'DICOM')  # Watch out before running this
            break
    
    os.mkdir(os.path.join(output_folder,patient))
    try:
        convert_rtstruct(
            dcm_path,
            rts_path,
            prefix = f'RTSTRUCT_{patient}_',
            output_dir = os.path.join(output_folder,patient),
            output_img = patient
        )
        print(f"Proceed {patient}")
    except:
        print(f"Fail to extract {patient}")     
    return

In [ ]:
for patient in os.listdir(img_folder):
    label = {}
    try:
        # Get path of image and ROI
        dcm_path = os.path.join(img_folder,patient)
        rts_path = os.path.join(roi_folder,patient)
    except:
        continue

    #get ID as tuple
    for rts in os.listdir(rts_path):
        get = rts.split('_')
        label[int(get[-1][1:])]= rts
        
    file = max(label) #coronal   
    rts_path = os.path.join(rts_path,label[file],'RTSTRUCT')
    rts_path = os.path.join(rts_path,os.listdir(rts_path)[0])
    param = {
        "file": file,
        "patient": patient,
        "dcm_path": dcm_path,
        "rts_path": rts_path,
        "output_folder": output_folder
    }
    convert(**param) #convert coronal Dicom and RTSTRUCT to Nifti


Another setting which can be useful is to also output the image series as a NIfTI file into the output directory. You can also specify the pixel spacing of the output:

In [7]:
img_out = 'TEST'
output_path = Path("../testcase/DCMRT_ROI")
output_path.mkdir(parents=True, exist_ok=True)

dcm_path = dcm_axl_path
rts_path = cor_path

convert_rtstruct(
    dcm_path,
    rts_path,
    prefix = 'RTSTRUCT_',
    output_dir= output_path,
    output_img=img_out
)